In [337]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [338]:
matches1 = pd.read_csv("assets/2022-2023.csv")
matches2 = pd.read_csv("assets/2023-2024.csv")
matches3 = pd.read_csv("assets/2024-2025.csv")
matches = pd.concat([matches1, matches2, matches3])

In [339]:
matches["Date Time (US Eastern)"] = pd.to_datetime(matches["Date Time (US Eastern)"])

/var/folders/cn/j21vzthj6ds5gjqdtwlgj4f00000gn/T/ipykernel_943/2735601536.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  matches["Date Time (US Eastern)"] = pd.to_datetime(matches["Date Time (US Eastern)"])


In [340]:
#advantages and disadvantages to consider when teams play against each other

matches["homeAway_code"] = np.where(matches["Team"] == matches["Home Team"], 0, 1) 

matches["opponent_code"] = np.where(
    matches["Team"] == matches["Home Team"],
    matches["Away Team"].astype("category").cat.codes,
    matches["Home Team"].astype("category").cat.codes
)
matches["day_code"] = matches["Date Time (US Eastern)"].dt.dayofweek
matches["date"] = matches["Date Time (US Eastern)"].dt.date

In [341]:
#determine the match result
matches["target"] = np.where(
    matches["Home Goal"] == matches["Away Goal"], 2,  
    np.where(
        (matches["Home Goal"] > matches["Away Goal"]) & (matches["Team"] == matches["Home Team"]), 1,  
        np.where(
            (matches["Home Goal"] > matches["Away Goal"]) & (matches["Team"] != matches["Home Team"]), 0,  
            np.where(
                (matches["Home Goal"] < matches["Away Goal"]) & (matches["Team"] == matches["Away Team"]), 1,  
                0  
            )
        )
    )
)

In [342]:
#began using machine learning 
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [ ]:
predictors = ["homeAway_code", "opponent_code", "day_code"]

In [343]:
train = matches[matches["Date Time (US Eastern)"] <= '2024-05-01']

In [344]:
test = matches[matches["Date Time (US Eastern)"] > '2024-05-01']

In [345]:
rf.fit(train[predictors], train["target"])

KeyError: "['shotsOnTarget_rolling', 'accuratePasses_rolling', 'accurateCrosses_rolling', 'blockedShots_rolling', 'effectiveTackles_rolling', 'effectiveClearance_rolling'] not in index"

In [315]:
preds = rf.predict(test[predictors])

In [316]:
accuracy = accuracy_score(test["target"], preds)
accuracy

0.4722222222222222

In [317]:
combine = pd.DataFrame(dict(actual=test["target"], prediction=preds))

In [318]:
pd.crosstab(index=combine["actual"], columns=combine["prediction"])

prediction,0,1,2
actual,,,
0,67,32,11
1,36,60,14
2,27,32,9


In [319]:
precision_score(test["target"], preds, average='weighted', zero_division=0)

0.4441601185552798

In [320]:
grouped_matches = matches.groupby("Team")

In [321]:
columns = ["shotsOnTarget", "accuratePasses", "accurateCrosses",  
           "blockedShots", "effectiveTackles", "effectiveClearance"]
new_columns = [f"{c}_rolling" for c in columns]

In [322]:
""" 
This function uses the stats from the previous 3 games 
to determine the team's condition and predict the next game. 
"""
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("Date Time (US Eastern)")
    rolling_stats = group[cols].shift(1).rolling(window=3).mean()
    for col, new_col in zip(cols, new_cols):
            group[new_col] = rolling_stats[col]
    group = group.dropna(subset=new_cols)
    return group

In [323]:
matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, columns, new_columns))
matches_rolling.droplevel("Team")
matches_rolling = matches_rolling.reset_index(drop=True)
matches_rolling.index = range(matches_rolling.shape[0])

/var/folders/cn/j21vzthj6ds5gjqdtwlgj4f00000gn/T/ipykernel_943/688239438.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, columns, new_columns))


In [324]:
def make_predictions(data, predictors):
    train = data[data["Date Time (US Eastern)"] <= '2024-05-01']
    test = data[data["Date Time (US Eastern)"] > '2024-05-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combine = pd.DataFrame(dict(actual=test["target"], prediction=preds), index=test.index)
    precision = precision_score(test["target"], preds, average='macro', zero_division=0)
    return combine, precision

In [325]:
predictors = ["homeAway_code", "opponent_code", "day_code"] + new_columns
combined, precision = make_predictions(matches_rolling, predictors)
combined = combined.merge(matches_rolling[["date", "Team" ,"Home Team", "Away Team", "target"]], left_index=True, right_index=True)

combined_liverpool = combined[combined["Team"] == "Liverpool"]
combined_liverpool

,actual,prediction,date,Team,Home Team,Away Team,target
950,1,1,2024-05-05,Liverpool,Liverpool,Tottenham Hotspur,1
951,2,1,2024-05-13,Liverpool,Aston Villa,Liverpool,2
952,1,1,2024-05-19,Liverpool,Liverpool,Wolverhampton Wanderers,1
953,1,1,2024-08-17,Liverpool,Ipswich Town,Liverpool,1
954,1,1,2024-08-25,Liverpool,Liverpool,Brentford,1
955,1,1,2024-09-01,Liverpool,Manchester United,Liverpool,1
956,0,1,2024-09-14,Liverpool,Liverpool,Nottingham Forest,0
957,1,1,2024-09-21,Liverpool,Liverpool,AFC Bournemouth,1
958,1,1,2024-09-28,Liverpool,Wolverhampton Wanderers,Liverpool,1
959,1,1,2024-10-05,Liverpool,Crystal Palace,Liverpool,1


In [326]:
accuracy

0.4722222222222222

In [327]:
precision

0.41468472280998725